In [2]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [3]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [4]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


An NYC-based robotic company and kitchen solutions provider, Aniai, recently closed a significant $12M Pre-Series A funding round. The funding was led by InterVest, with participation from SV Investment, UK-based Ignite Innovation, and existing investor Capstone Partners. This capital injection is intended to support the launch of Aniai's first dedicated manufacturing facility, Factory One, in South Korea. The establishment of this facility will help ensure stable production and efficient supply chain management for Aniai's flagship product, the 'Alpha Grill.' The company's strategic move aligns with its ongoing expansion efforts in North America and globally.

Established in 2020 and under the leadership of CEO Gunpil Hwang, Aniai specializes in developing robotic solutions for cooking hamburgers. The Alpha Grill, their main product, utilizes robotics to enhance the efficiency of hamburger cooking. It features a double-sided grill capable of cooking up to eight patties simultaneously 

In [5]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [8]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt_mom_test_problem = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the mom-test method: 
        
        The mom-test is a strategy for getting insights out of potential users, even when said potential users may be prone to lying to you. You mom is the example in this strategy: she will say she likes something because it’s you that’s asking.

        You don’t ask your mom whether she likes your idea for a new cookbook app. You ask her how, when, and why she uses her device, and about her current cookbooks. The question about your idea will likely lead to a false answer, while the questions about her activities (a.k.a Mom Test questions) will give you insight into what she actually likes and wants from a product.

        In essence, you’re getting to the bottom of what the user actually wants, rather than trying to force your idea down their throat. You’re listening to the user without misinterpreting what they’re saying.

        When a customer makes a request, get them to explain exactly how and why that would help them.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
mom_test_problems = []

user_prompt_mom_test_problem = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Please identify problems that are experienced by customers in the above text and answer the following mom-test questions from the perspective of one of these customers:

    - Why do you want that?
    - What would that empower you to do?
    - How are you managing without it?
    - How would that fit into your day?
    - Do you think we should push back the launch to add the feature, or is it something we could include at a later date?
    - See how these questions lead the user to give more detailed feedback, which in turn creates more powerful research data for you?

    Please provide some sources of inspiration that you used from the document excerpts


    
    """
    
completion_mom_test = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt_mom_test_problem},
            {"role": "user", "content": user_prompt_mom_test_problem}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
mom_test_problems.append(completion_mom_test.choices[0].message.content)
print(completion_mom_test.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


# Next agent takes mom-test output and comes up with solutions based on this
mom_test_solutions = []

system_prompt_mom_test_solution = f"""
    You are a highly qualified CTO with decades of experience in coming up with novel, exciting business opportunities.
"""

user_prompt_mom_test_solution = f"""

    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {mom_test_problems}
    ```    
  
    Please develop 5 really specific ideas/visions on how the problems outlined in the are delimited by ''' can be solved. The solutions should consider the following points:
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    For the solution, please write only 40 words. It should contain the following contents:
    1) A title for the solution in max 3 words
    2) Your solution summarized in 1 sentence
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) How the solutions is novel, that is, not already existing in the market
    5) Please provide some sources of inspiration that you used from the document excerpts.

    """
    

completion_mom_test_solution = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_mom_test_solution},
{"role": "user", "content": user_prompt_mom_test_solution}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion_mom_test_solution.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
mom_test_solutions.append(completion_mom_test_solution.choices[0].message.content)


#Summary

summary_solutions = []

user_prompt_summarizer = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas:
    {mom_test_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (5 words max)
    “Solved Problem”: 1 sentence summary of the problem that is being solved 
    “Solution”: Provide a short summary of the solution and how it works (3 sentences max)
    “Impact”: How does the solution solve the specific problem (3 sentences max)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (5 sentences max)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion_summarizer = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_mom_test_solution},
{"role": "user", "content": user_prompt_summarizer}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion_summarizer.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions.append(completion_summarizer.choices[0].message.content)



Given the context of the problem outlined - lack of seamless integration and communication between different factory automation systems and equipment - let's assume we're speaking with a factory automation engineer from one of the companies mentioned, for instance, Software Defined Automation. This company is focusing on revolutionizing factory automation through its Industrial-Control-as-a-Service (ICaaS) offering, addressing issues like proprietary silos in control technology stacks.

### Why do you want that?

"As a factory automation engineer, seamless integration across different systems is crucial. It minimizes downtime and ensures that our production lines are running as efficiently as possible. The current lack of integration means we're often troubleshooting issues that arise from these silos, leading to costly delays."

### What would that empower you to do?

"Having a seamless integration would significantly reduce the time we spend on manual interventions and troubleshootin

In [9]:
# Initial rating of the solutions

initial_rating = []

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Please take all solutions from the following text: {summary_solutions}
First, repeat the following points for the first solution:
    - "Name"
    - “Solved Problem”
    - “Solution”
    - “Impact”
    - “Technology”
    - “Sources”

Then, rate the first solution according to the following criteria. Please justify your ratings in max one sentence. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)
    - Overall Rating: Sum of all the previous ratings, out of 15

Now, repeat these steps for all solutions.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability




["**Dynamic Integration Fabric**\n\n- **Solved Problem**: Addresses the issue of static and inefficient communication paths between diverse factory systems.\n- **Solution**: This cloud service dynamically optimizes integration paths between factory systems in real-time, ensuring efficient data flow and system communication without manual intervention.\n- **Impact**: By continuously adapting to changes in system load and efficiency requirements, it eliminates the need for manual troubleshooting, significantly reducing downtime and increasing overall system efficiency.\n- **Technology**: Utilizes advanced cloud computing resources and algorithms to monitor system performance metrics in real-time, applying machine learning to predict and automatically adjust communication pathways based on current and anticipated system states. This approach leverages Internet of Things (IoT) technology to gather data from factory systems, ensuring a comprehensive understanding of the integration landscap